In [26]:
import os
import torch
from torch import nn
from poutyne import Model, CSVLogger
from poutyne.framework import ModelCheckpoint, EarlyStopping, plot_history
import numpy as np
import torchmetrics
from datetime import datetime
import sys
import pandas as pd
from custom_lib.custom_models.basic_nn import NeuralNetwork
from custom_lib.data_prep import data_transformation_pipeline, data_loader
import matplotlib as plt
import torchvision.models as models
import time


ImportError: cannot import name 'define_custom_eff_net' from 'custom_lib.custom_models.basic_nn' (/Users/ayw1327/Documents/GitHub/cxr_thesis/custom_lib/custom_models/basic_nn.py)

In [21]:
# Tuneable Params
lr = 1e-3

data_dir = "data"

# Define a model name (e.g., "model1")
model_name = "custom_b0"

save_logs = True

epochs = 1

image_size = 224
rotate_angle=None
horizontal_flip_prob=None
brightess_contrast=None
gaussian_blur=None
normalize=False
seed = 42



In [22]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using mps device


In [23]:
train_transform = data_transformation_pipeline(image_size = image_size,
                                               rotate_angle=rotate_angle,
                                               horizontal_flip_prob=horizontal_flip_prob,
                                               gaussian_blur=gaussian_blur,
                                               normalize=normalize,
                                               is_train=True)
test_transform = data_transformation_pipeline(image_size = image_size,
                                               rotate_angle=rotate_angle,
                                               horizontal_flip_prob=horizontal_flip_prob,
                                               gaussian_blur=gaussian_blur,
                                               normalize=normalize,
                                               is_train=False)
val_transform = data_transformation_pipeline(image_size = image_size,
                                               rotate_angle=rotate_angle,
                                               horizontal_flip_prob=horizontal_flip_prob,
                                               gaussian_blur=gaussian_blur,
                                               normalize=normalize,
                                               is_train=False)

train_loader , val_loader, test_loader, num_classes = data_loader(data_dir, 
                                                     train_transform=train_transform,
                                                     test_transform=test_transform,
                                                     val_transform=val_transform,
                                                     seed=seed
                                                     )



Train size: 6177, Validation size: 772, Test size: 773


In [24]:
# If Else statement to determine if the user has passed a custom model or a prebuilt model.
# If the model_name contains the word custom, the code extracts the version letter and number
# and passes the proper configuration to the model
if ("custom" in model_name):
    from custom_lib.custom_models.custom_eff_net import define_custom_eff_net
    import re

    efficient_net_config = {
        # tuple of width multiplier, depth multiplier, resolution, and Survival Prob for
        # each efficientnet version
        "b0" : (1.0, 1.0, 224, 0.2),
        "b1" : (1.0, 1.1, 240, 0.2),
        "b2" : (1.1, 1.2, 260, 0.3),
        "b3" : (1.2, 1.4, 300, 0.3),
        "b4" : (1.4, 1.8, 380, 0.4),
        "b5" : (1.6, 2.2, 456, 0.4),
        "b6" : (1.8, 2.6, 528, 0.5),
        "b7" : (2.0, 3.1, 600, 0.5)
    }

    define_custom_eff_net(efficient_net_config=efficient_net_config, num_classes=num_classes)

else:
    model_class = getattr(models, model_name, None)

    if model_class is None:
        raise ValueError(f"Model '{model_name}' is not available in torchvision.models.")

    # Initialize the model
    model = model_class(pretrained=True)











# Compound scaling factors for efficient-net family.


# 6. Wrap the model with Poutyne
poutyne_model = Model(
    model,
    optimizer=torch.optim.Adam(model.parameters(), lr=lr),
    loss_function=nn.CrossEntropyLoss(),
    batch_metrics=['accuracy'],
    device=device
)

In [25]:

callbacks = None

# if save_logs == True:
#     # Callback: Save the best model based on validation accuracy
#     checkpoint = ModelCheckpoint(f"{results_dir}/best_model.pth", monitor='val_loss', mode='min', save_best_only=True)

#     # Callback: Stop training early if validation accuracy doesn't improve for 5 epochs
#     early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=5)

#     # Set up the logger
#     csv_logger = CSVLogger(f"{results_dir}/training_logs.csv")

#     callbacks = [checkpoint, early_stopping, csv_logger]

start_time = time.time()
# 7. Train the model
history = poutyne_model.fit_generator(train_loader, val_loader, epochs=epochs, verbose=True,
                            callbacks = callbacks)
end_time = time.time()

run_time = end_time - start_time

print(f"Model training took {run_time / 60} minutes")


Epoch: 1/1 Step: 194/194 100.00% |████████████████████|ETA: 0.00s loss: 1.381286 acc: 0.000000      

KeyboardInterrupt: 

In [9]:
# 8. Evaluate the model
test_metrics = poutyne_model.evaluate_generator(test_loader)
print("Test metrics:", test_metrics)

Test steps: 13 28.58s test_loss: 0.307702 test_acc: 89.133247                                  
Test metrics: (0.3077017245729867, 89.13324708926261)
